# Forrestania: Gravity & Magnetics (Pure Python)

```{figure} ./images/landing.png
---
scale: 30%
align: right
---
```

This case study focuses on the standalone and joint inversion of airborne magnetic data and synthetic gravity data. The magnetic data were downloaded from the [Geoscience Australia GADDS Portal](https://portal.ga.gov.au/persona/gadds), while the gravity data were generated via forward modelling specifically for this exercise based on a derived iso-shell from magnetic inversion results. We cover the following steps:

- [Data imports and pre-processing](forrestania-data)
- [Standalone inversion of gravity data](forrestania-gravity)
- [Standalone inversion of magnetic data](forrestania-magnetics)
- [Joint cross-gradient inversion](forrestania-joint)
- [K-means clustering analysis](forrestania-kmeans)

```{note}
The steps taken in this tutorial are strongly based on the official [SimPEG Tutorials](https://simpeg.xyz/user-tutorials/).
```

In [ ]:
# Import necessary Python libraries for data handling and visualization
import os
import zipfile
from pathlib import Path
from tempfile import mkdtemp

import discretize
import matplotlib.pyplot as plt
import numpy as np
import pandas
import scipy as sp
import simpeg
from geoh5py import Workspace, objects
from PIL import Image

# Import SimPEG library
from simpeg import (
    dask,  # Parallel version of the code
    potential_fields,
)

# Mira Geoscience specific libraries
from simpeg_drivers import assets_path

## Geological setting

The Forrestania Greenstone Belt, located in the Youanmi Terrane of the Eastern Yilgarn Craton, hosts significant Ni-Cu-PGE mineralisation, primarily within komatiitic units (Frost, 2003; Collins et al., 2012). The belt is structurally complex, featuring synclines, faults, and shear zones that play a key role in sulphide localisation. It is bounded by granitic and gneissic basement, intruded by monzogranite and granodiorite, and transected by Proterozoic dolerite dykes (Perring et al., 1995). Multiple deformation events have remobilised nickel sulphides from their original host rocks into footwall sediments and granitic intrusions, resulting in both typical and atypical mineralisation styles, including granite-hosted sulphides (Collins et al., 2012).

The study area is underlain mainly by granitic rocks, forming the basement to the Parker-Range–Hatters Hill greenstone belt immediately to the east. This N–S-trending belt is the main regional host of nickel sulphide deposits, which occur within narrow ultramafic units in an Archaean sequence of metabasalts and sulphidic sediments. Notable nearby deposits include Beautiful Sunday, Flying Fox, and New Morning.

A prominent high-magnetic anomaly, trending E–W to ENE–WSW, lies immediately west of the greenstone belt. The anomaly’s orientation, strength, and coincident elevated nickel-in-soil geochemistry suggest it may represent a mafic intrusion—making it a key target for the current geophysical inversion.

(forrestania-data-os)=
## Data Preparation

This step involves importing and formatting the necessary datasets for inversion using Mira Geoscience's libraries. For your own inversion projects, you may streamline the data preparation process by using standard Python libraries, depending on your specific needs.

### Download and unzip the dataset

We first need to unzip the package and import the data into a usable format.

The zipped package contains the following three files:
 - Airborne magnetic survey: `60472_AOI4.csv`
 - Ground gravity survey: `Forrestania_Gravity_Station_trim_.csv`
 - Digital Elevation Model (DEM): `Forrestania_SRTM1 Australia_MGA50.tiff`

In [ ]:
# Create a temporary directory for extracting the zip contents
temp_dir = Path(mkdtemp())

# Use the `assets_path()` from Mira Geoscience's simpeg_drivers to download and locate the Forrestania dataset
# This gives us the full path to the zip file we want to extract
file = assets_path() / r"Case studies/Forrestania_SRTM1 Australia_MGA50_CSV.zip"

# Print the resolved path to confirm where the file is located on disk
print(f"Dataset path: {file}")

# Extract all contents of the zip file into the temporary directory
with zipfile.ZipFile(file, "r") as zf:
    zf.extractall(temp_dir)

# List all the files that were extracted
files = list(temp_dir.iterdir())

### Load the CSV and geotiff files
We will use the `pandas` library to read the CSV files and Mira Geoscience's `geoh5py` library to read the geotiff file. 

In [ ]:
grav_survey = pandas.read_csv(next(file for file in files if "Gravity" in file.name))

### Processing Elevation Data

#### Step 1: Convert the geoImage to a 2D Grid with values

We will utilize the functionality made available in the `geoh5py` library to read and convert the geotiff to a gridded DEM with geographic coordinates.

In [ ]:
# Load the elevation GeoTIFF using geoh5py
ws = Workspace()
geotiff = objects.GeoImage.create(
    ws, image=str(next(file for file in files if "SRTM1" in file.name))
)

# Register the geospatial reference system from the TIFF metadata using geoh5py
geotiff.georeferencing_from_tiff()

# Convert the image into a grid object (2D array of elevations) in geoh5py
grid = geotiff.to_grid2d()
elevations = grid.children[0].values  # Extract elevation values

# Visualise the raw elevation data (some values may be small placeholders, zeros or NaNs)
fig, ax = plt.subplots(figsize=(10, 6))
elev_image = elevations.reshape(grid.shape, order="F").T
im = ax.imshow(elev_image, cmap="terrain", origin="lower")

cbar = plt.colorbar(im, orientation="horizontal", pad=0.1)
cbar.set_label("Raw Elevation (m)")

ax.set_title("Raw Elevation Grid (including placeholders or NaNs)")
ax.set_xlabel("Grid X")
ax.set_ylabel("Grid Y")
plt.tight_layout()
plt.show()

#### Clean Up Invalid Elevation Values
We also need to remove the zeros from the rotated DEM.

In [ ]:
# Check minimum elevation value
print(f"Minimum elevation value: {np.nanmin(elevations)}")

In [ ]:
# Some elevation values are tiny placeholders for "no data", replace with NaN
logic = np.abs(elevations) < 2e-38
elevations = np.where(logic, np.nan, elevations)

# Plot cleaned-up elevation map
fig, ax = plt.subplots(figsize=(10, 6))
cleaned_elev_image = elevations.reshape(grid.shape, order="F").T
im = ax.imshow(cleaned_elev_image, cmap="terrain", origin="lower")

cbar = plt.colorbar(im, orientation="horizontal", pad=0.1)
cbar.set_label("Cleaned Elevation (m)")

ax.set_title("Cleaned Elevation Grid (NaNs filtered)")
ax.set_xlabel("Grid X")
ax.set_ylabel("Grid Y")
plt.tight_layout()
plt.show()

In [ ]:
# Our final DEM DEM array: [X, Y, Elevation]
dem = np.c_[grid.centroids[:, :2], elevations]

### Magnetic Data Processing

#### Step 1: Extract the magnetic data and format

In [ ]:
# Load magnetic and gravity CSVs with pandas
mag_dataframe = pandas.read_csv(next(file for file in files if "AOI4" in file.name))

# Preview the dataset
print("Magnetic survey data preview:")
mag_dataframe

#### Step 2: Attach elevation to the airborne magnetic data

There are a lot more information that we need from the original CSV, but also some that are missing.
Absolute elevations for this survey are currently not available, only the height above the DEM. So we will need to extract it from the grid.
The fastest interpolation is a nearest neighbour.

In [ ]:
# Extract relevant columns: projected easting (X), northing (Y), and radar altimeter (height above terrain)
mag_locs = mag_dataframe[["X_MGA50", "Y_MGA50", "RADALT"]].to_numpy()

# Build a KDTree for fast spatial search on the DEM 2D coordinates
tree = sp.spatial.cKDTree(dem[:, :2])  # Only XY from DEM
_, ind = tree.query(mag_locs[:, :2])  # Nearest neighbour indices from DEM

# Add DEM ground elevation to RADALT to get true elevation
mag_locs[:, 2] += dem[ind, 2]

# Check elevation range
print(f"Maximum interpolated elevation: {np.nanmax(mag_locs[:, 2])}")

In [ ]:
# Plot the survey points coloured by interpolated elevation
fig, ax = plt.subplots(figsize=(10, 6))
sc = ax.scatter(
    mag_locs[:, 0],
    mag_locs[:, 1],  # X and Y coordinates
    c=mag_locs[:, 2],  # Color is based on elevation
    marker="o",  # Circle markers
    cmap="terrain",
    s=4,  # Size of points slightly larger for visibility
    edgecolors="none",
)

cbar = plt.colorbar(sc, orientation="horizontal", pad=0.1)
cbar.set_label("Interpolated Elevation (m)")

ax.set_aspect("equal")
ax.set_title("Magnetic Flight Path with Interpolated Elevation")
ax.set_xlabel("Easting (MGA50)")
ax.set_ylabel("Northing (MGA50)")
plt.tight_layout()
plt.show()

#### Step 3: Compute residual data

The source of the magnetic signal can generally be attributed to the presence or destruction of magnetic minerals in rocks (mainly magnetite). Magnetometers measure the Total Magnetic Intensity (TMI), which includes both the primary (source) field and secondary fields (signal) from the local geology. 

The inversion routine requires Residual Magnetic Intensity (RMI). The first step is to compute and remove the primary field (IGRF) at the time and location of acquisition.  The survey was conducted between February and March 1988.

##### Lookup/remove IGRF 

Information about the inducing field strength at the time and location of the survey can be accessed from the [NOAA site](https://www.ngdc.noaa.gov/geomag/calculators/magcalc.shtml?useFullSite=true#igrfwmm).

The **inducing field parameters** calculated at the time and location of the survey are

```{figure} ./images/NOAA_IGRF.png
---
scale: 20%
---
[Click to enlarge]
```

- Magnitude: 59294 nT
- Inclination: -67.1$\degree$
- Declination: -0.89$\degree$  

In [ ]:
# Reduce the data by the inducing field strength
mag_data = mag_dataframe["MAGCOMP"] - 59294
fig, ax = plt.subplots()
im = plt.hist(mag_data, bins=100)
ax.set_aspect(1)
print(f"Median: {mag_data.median()} nT")

#### Step 4: Detrend
The local background field appears to be slightly lower (~283 nT) than the computed IGRF model, as most of the data away from the main anomaly are below 0. To avoid modelling this background trend, we can remove the median value. 

In [ ]:
mag_data -= mag_data.median()

fig, ax = plt.subplots()
im = plt.hist(mag_data, bins=100)
ax.set_aspect(1)
print(f"Median: {mag_data.median()} nT")

#### Step 5: Downsampling

We can downsample the data along lines to reduce the computation cost of the inversion. Since the average flight height of this survey was 60 m, we can confidently downsample the lines by the same amount without affecting the wavelengths contained in the data.

We can do this fairly efficiently by sampling the data over a regular grid.

In [ ]:
x_grid, y_grid = np.meshgrid(
    np.arange(745500, 748500, 60),
    np.arange(6415500, 6418500, 60),
)

# Generate a KDTree from the survey
tree = sp.spatial.cKDTree(mag_locs[:, :2])

# Find the nearest indices for each grid points
_, ind = tree.query(np.c_[x_grid.flatten(), y_grid.flatten()])

mag_survey = np.c_[mag_locs, mag_data][ind, :]
print(mag_survey.shape)

In [ ]:
# Plot the survey points coloured by interpolated elevation
fig, ax = plt.subplots(figsize=(10, 6))
sc = ax.scatter(
    mag_survey[:, 0],
    mag_survey[:, 1],  # X and Y coordinates
    c=mag_survey[:, -1],  # Color is based on elevation
    marker="o",  # Circle markers
    cmap="rainbow",
    s=10,  # Size of points slightly larger for visibility
    edgecolors="none",
)

cbar = plt.colorbar(sc, orientation="horizontal", pad=0.1)
cbar.set_label("Magnetic data (nT)")

ax.set_aspect("equal")
ax.set_title("Residual (detrended) data downsampled at 60 m")
ax.set_xlabel("Easting (MGA50)")
ax.set_ylabel("Northing (MGA50)")
plt.tight_layout()
plt.show()

(forrestania-gravity-os)=
## Gravity data processing

The last dataset to format is the gravity survey. In this case, the survey is already located at the surface, and the data is already provided as terrain corrected (2.67 g/cc) in mGal. No extra transformation is required. 

In [ ]:
# Read the two CSVs with pandas
grav_dataframe = pandas.read_csv(next(file for file in files if "Gravity" in file.name))
grav_dataframe

In [ ]:
grav_survey = grav_dataframe.to_numpy()


fig, ax = plt.subplots(figsize=(10, 6))
sc = ax.scatter(
    grav_survey[:, 0],
    grav_survey[:, 1],  # X and Y coordinates
    c=grav_survey[:, -1],  # Color is based on elevation
    marker="o",  # Circle markers
    cmap="RdBu_r",
    s=10,  # Size of points slightly larger for visibility
    edgecolors="none",
)

cbar = plt.colorbar(sc, orientation="horizontal", pad=0.1)
cbar.set_label("Gravity data (mGal)")

ax.set_aspect("equal")
ax.set_title("Terrain corrected (2.67 g/cc) gravity data")
ax.set_xlabel("Easting (MGA50)")
ax.set_ylabel("Northing (MGA50)")
plt.tight_layout()
plt.show()

## Inversion

### Magnetic

We begin our analysis with the airborne magnetic survey. Our goal is to model the shape and position of magnetized geological units in 3D.

#### Create a mesh 

We need to break down the subsurface into a grid of cells under the following considerations: use the least number of cells to remain computationally efficient while having enough resolution to model small features accurately. We can achieve both goals with an octree mesh.

In [ ]:
# Determine the smallest cell size, half the flight height and data resolution
base_cells = [30, 30, 30]

# Create the base TreeMesh grid (without refinement)
mag_octree = discretize.utils.mesh_builder_xyz(
    mag_survey[:, :3],
    base_cells,
    mesh_type="tree",
    depth_core=2000,  # At least as deep as 2000 m
)


# Refine around each receiver location
mag_octree.refine_points(
    mag_survey[:, :3],
    level=-1,  # Use the (last) highest level
    padding_cells_by_level=[6, 6, 6],  # Number of cells at 30 m, 60 m and 120 m
    finalize=False,
)

# Refine along topography
mag_octree.refine_surface(
    dem,
    level=-3,  # Only refine at 120 m on dem
    padding_cells_by_level=[1],
    finalize=True,  # Complete the mesh on our last call
)

#### Defining the air-ground domains

We need to tell the inversion what part of the octree grid lies below the surface. 

In [ ]:
active = discretize.utils.active_from_xyz(mag_octree, dem)
n_actives = int(active.sum())
print(f"Number of active cells: {n_actives}")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
mag_octree.plot_slice(active, normal="Y", grid=True, ax=ax)
ax.set_xlim([744500, 749500])
ax.set_aspect(1)

#### Defining a survey and simulation

We now define the "geophysical" experiment in terms of survey configurations, and tell SimPEG how to simulate data. 

In [ ]:
# Create a survey with receivers
receiver_list = potential_fields.magnetics.receivers.Point(
    mag_survey[:, :3], components=["tmi"]
)

# Define the inducing field parameter ("the source")
source_field = potential_fields.magnetics.sources.UniformBackgroundField(
    receiver_list=receiver_list,
    amplitude=59294,
    inclination=-67.1,
    declination=-0.89,
)

# Define the survey
survey = potential_fields.magnetics.survey.Survey(source_field)

# Put it all together
simulation = potential_fields.magnetics.simulation.Simulation3DIntegral(
    survey=survey,
    mesh=mag_octree,
    active_cells=active,
    chiMap=simpeg.maps.IdentityMap(nP=n_actives),
)

#### Convert to a Simpeg Data

We can create a SimPEG survey that defines the position, data type and specs about the inducing field.

In [ ]:
# Create a Data class that contains everything
data = simpeg.Data(
    survey,
    dobs=mag_survey[:, -1],
    standard_deviation=50,  # Assign the data uncertainty to a quarter of the standard deviation
)

#### Define the data misfit function

One of main requirement for the inversion is to fine a model that can replicate the observed data. We enforce that constraint with a least-square measure of data fit.

In [ ]:
data_misfit = simpeg.data_misfit.L2DataMisfit(data=data, simulation=simulation)

#### Create the regularization function

In order to constrain the model, we need to add a `regularization` function. We will use a function that can promote sparsity.

In [ ]:
regularization = simpeg.regularization.Sparse(
    mag_octree,
    active_cells=active,
    reference_model=np.zeros(n_actives),
    norms=[0, 2, 2, 2],
)

#### Define inversion directives

We want to control the different steps of the inversions, such as:
 - Initial trade-off parameter
 - Cooling schedule for data fit
 - Sparsity promoting iterations and target

This is done through `Directives`.

In [ ]:
sensitivity_weights = simpeg.directives.UpdateSensitivityWeights(every_iteration=False)
starting_beta = simpeg.directives.BetaEstimate_ByEig(beta0_ratio=10)
update_jacobi = simpeg.directives.UpdatePreconditioner(update_every_iteration=True)
update_irls = simpeg.directives.UpdateIRLS(max_irls_iterations=25)

directives_list = [
    update_irls,
    sensitivity_weights,
    starting_beta,
    update_jacobi,
]

#### Define the inverse problem

In [ ]:
# Define the optimization strategy
optimizer = simpeg.optimization.ProjectedGNCG(
    lower=0.0,  # Apply a lower bound to the model
    maxIter=25,
)

# Define an inverse problem, the inversion and run
inv_problem = simpeg.inverse_problem.BaseInvProblem(
    data_misfit, regularization, optimizer
)
inversion = simpeg.inversion.BaseInversion(inv_problem, directives_list)

# Create a simple starting model
m_start = np.ones(active.sum()) * 1e-4
rec_model = inversion.run(m_start)

#### Results

In [ ]:
# Validate observed versus predicted
fig = plt.figure(figsize=(12, 8))
ax = plt.subplot(1, 3, 1)
simpeg.utils.plot_utils.plot2Ddata(mag_survey[:, :2], mag_survey[:, -1], ax=ax)
ax.set_title("Observed")

ax = plt.subplot(1, 3, 2)
simpeg.utils.plot_utils.plot2Ddata(mag_survey[:, :2], inv_problem.dpred[0], ax=ax)
ax.set_title("Predicted")

ax = plt.subplot(1, 3, 3)
simpeg.utils.plot_utils.plot2Ddata(
    mag_survey[:, :2], mag_survey[:, -1] - inv_problem.dpred[0], ax=ax
)
ax.set_title("Residual")

In [ ]:
# Mapping to ignore inactive cells when plotting
plotting_map = simpeg.maps.InjectActiveCells(mag_octree, active, np.nan)

fig = plt.figure(figsize=(12, 12))

ax = plt.subplot(2, 2, 1)
im = mag_octree.plot_slice(
    plotting_map * inv_problem.l2model, normal="Z", ind=85, ax=ax, clim=[0, 0.25]
)
ax.set_xlim([745500, 748500])
ax.set_ylim([6415500, 6418500])
ax.set_aspect(1)
cbar = plt.colorbar(im[0], orientation="horizontal", pad=0.1)
cbar.set_label("Susceptibility (SI)")

ax = plt.subplot(2, 2, 3)
im = mag_octree.plot_slice(
    plotting_map * inv_problem.l2model, normal="Y", ind=70, ax=ax, clim=[0, 0.25]
)
ax.set_aspect(1)
ax.set_xlim([745500, 748500])
ax.set_ylim([-1000, 500])
ax.set_aspect(1)


ax = plt.subplot(2, 2, 2)
im = mag_octree.plot_slice(
    plotting_map * rec_model, normal="Z", ind=85, ax=ax, clim=[0, 2.0]
)
ax.set_xlim([745500, 748500])
ax.set_ylim([6415500, 6418500])
ax.set_aspect(1)
cbar = plt.colorbar(im[0], orientation="horizontal", pad=0.1)
cbar.set_label("Susceptibility (SI)")

ax = plt.subplot(2, 2, 4)
im = mag_octree.plot_slice(
    plotting_map * rec_model, normal="Y", ind=70, ax=ax, clim=[0, 2.0]
)
ax.set_aspect(1)
ax.set_xlim([745500, 748500])
ax.set_ylim([-1000, 500])
ax.set_aspect(1)

We note the following:

 - It took 9 iterations to converge to the target misfit.
 - The following 21 iterations have increased the model complexity ($phi_m$) while preserving the data fit ($phi_d$).
 - Small anomalies are under-fitted, which could warrant a second inversion with lower uncertainties.

The smooth solution (iteration 9) indicates the presence of a susceptible body at depth. The shape and location of the anomaly resemble the solution obtained with the gravity modelling, including the presence of smaller "fuzzy" anomalies at the margins.  

As an alternative solution, the compact model (iteration 23) recovers a well-defined dense body within a mostly uniform background. Susceptibility contrasts have substantially increased in the range of [0.0, 2.0] SI.

(forrestania-magnetics-os)=
### Gravity

We follow up with the processing of the ground gravity data.

In [ ]:
# Determine the smallest cell size, half the flight height and data resolution
base_cells = [30, 30, 30]

# Create the base TreeMesh grid (without refinement)
grav_octree = discretize.utils.mesh_builder_xyz(
    mag_survey[:, :3],  # Keep the same extent towards the joint
    base_cells,
    mesh_type="tree",
    depth_core=2000,  # At least as deep as 2000 m
)


# Refine around each receiver location
grav_octree.refine_points(
    grav_survey[:, :3],
    level=-1,  # Use the (last) highest level
    padding_cells_by_level=[6, 6, 6],  # Number of cells at 30 m, 60 m and 120 m
    finalize=False,
)

# Refine along topography
grav_octree.refine_surface(
    dem,
    level=-3,  # Only refine at 120 m on dem
    padding_cells_by_level=[1],
    finalize=True,  # Complete the mesh on our last call
)

#### Defining the air-ground domains

We need to tell the inversion what part of the octree grid lies below the surface. 

In [ ]:
active = discretize.utils.active_from_xyz(grav_octree, dem)
n_actives = int(active.sum())
print(f"Number of active cells: {n_actives}")

#### Defining a survey and simulation

We now define the "geophysical" experiment in terms of survey configurations, and tell SimPEG how to simulate data. 

In [ ]:
# Create a survey with receivers
receiver_list = potential_fields.gravity.receivers.Point(
    grav_survey[:, :3], components=["gz"]
)

# Define the inducing field parameter ("the source")
source_field = potential_fields.gravity.sources.SourceField(receiver_list=receiver_list)

# Define the survey
survey = potential_fields.gravity.survey.Survey(source_field)

# Put it all together
simulation = potential_fields.gravity.simulation.Simulation3DIntegral(
    survey=survey,
    mesh=grav_octree,
    active_cells=active,
    rhoMap=simpeg.maps.IdentityMap(nP=n_actives),
)

#### Convert to a Simpeg Data

We can create a SimPEG survey that defines the position, data type and specs about the inducing field.

In [ ]:
# Create a Data class that contains everything
data = simpeg.Data(
    survey,
    dobs=grav_survey[:, -1],
    standard_deviation=0.025,  # Assign the data uncertainty to a quarter of the standard deviation
)

#### Define the data misfit function

One of main requirement for the inversion is to fine a model that can replicate the observed data. We enforce that constraint with a least-square measure of data fit.

In [ ]:
data_misfit = simpeg.data_misfit.L2DataMisfit(data=data, simulation=simulation)

#### Create the regularization function

In order to constrain the model, we need to add a `regularization` function. We will use a function that can promote sparsity.

In [ ]:
regularization = simpeg.regularization.Sparse(
    grav_octree,
    active_cells=active,
    reference_model=np.zeros(n_actives),
    norms=[0, 2, 2, 2],
)

#### Define inversion directives

We want to control the different steps of the inversions, such as:
 - Initial trade-off parameter
 - Cooling schedule for data fit
 - Sparsity promoting iterations and target

This is done through `Directives`.

In [ ]:
sensitivity_weights = simpeg.directives.UpdateSensitivityWeights(every_iteration=False)
starting_beta = simpeg.directives.BetaEstimate_ByEig(beta0_ratio=10)
update_jacobi = simpeg.directives.UpdatePreconditioner(update_every_iteration=True)
update_irls = simpeg.directives.UpdateIRLS(max_irls_iterations=25)

directives_list = [
    update_irls,
    sensitivity_weights,
    starting_beta,
    update_jacobi,
]

#### Define the inverse problem

In [ ]:
# Define the optimization strategy
optimizer = simpeg.optimization.ProjectedGNCG(maxIter=25)

# Define an inverse problem, the inversion and run
inv_problem = simpeg.inverse_problem.BaseInvProblem(
    data_misfit, regularization, optimizer
)
inversion = simpeg.inversion.BaseInversion(inv_problem, directives_list)

# Create a simple starting model
m_start = np.ones(active.sum()) * 1e-4
rec_model = inversion.run(m_start)

#### Results

In [ ]:
# Validate observed versus predicted
fig = plt.figure(figsize=(12, 8))
ax = plt.subplot(1, 3, 1)
simpeg.utils.plot_utils.plot2Ddata(grav_survey[:, :2], grav_survey[:, -1], ax=ax)
ax.set_title("Observed")

ax = plt.subplot(1, 3, 2)
simpeg.utils.plot_utils.plot2Ddata(grav_survey[:, :2], inv_problem.dpred[0], ax=ax)
ax.set_title("Predicted")

ax = plt.subplot(1, 3, 3)
simpeg.utils.plot_utils.plot2Ddata(
    grav_survey[:, :2], grav_survey[:, -1] - inv_problem.dpred[0], ax=ax
)
ax.set_title("Residual")

In [ ]:
# Mapping to ignore inactive cells when plotting
plotting_map = simpeg.maps.InjectActiveCells(grav_octree, active, np.nan)

fig = plt.figure(figsize=(12, 12))

ax = plt.subplot(2, 2, 1)
im = grav_octree.plot_slice(
    plotting_map * inv_problem.l2model, normal="Z", ind=85, ax=ax, clim=[-0.1, 0.05]
)
ax.set_xlim([745500, 748500])
ax.set_ylim([6415500, 6418500])
ax.set_aspect(1)
cbar = plt.colorbar(im[0], orientation="horizontal", pad=0.1)
cbar.set_label("Density constrast (g/cc)")

ax = plt.subplot(2, 2, 3)
im = grav_octree.plot_slice(
    plotting_map * inv_problem.l2model, normal="Y", ind=70, ax=ax, clim=[-0.1, 0.05]
)
ax.set_aspect(1)
ax.set_xlim([745500, 748500])
ax.set_ylim([-1000, 500])
ax.set_aspect(1)


ax = plt.subplot(2, 2, 2)
im = grav_octree.plot_slice(
    plotting_map * rec_model, normal="Z", ind=85, ax=ax, clim=[-0.5, 0.05]
)
ax.set_xlim([745500, 748500])
ax.set_ylim([6415500, 6418500])
ax.set_aspect(1)
cbar = plt.colorbar(im[0], orientation="horizontal", pad=0.1)
cbar.set_label("Density constrast (g/cc)")

ax = plt.subplot(2, 2, 4)
im = grav_octree.plot_slice(
    plotting_map * rec_model, normal="Y", ind=70, ax=ax, clim=[-0.5, 0.05]
)
ax.set_aspect(1)
ax.set_xlim([745500, 748500])
ax.set_ylim([-1000, 500])
ax.set_aspect(1)

We note the following:

 - It took 13 iterations to converge to the target misfit.
 - The following 12 iterations have increased the model complexity ($phi_m$) but preserved the data fit ($phi_d$).
 - Most of the important gravity anomalies appear to be reproduced by the final model.

The smooth solution (iteration 10) indicates the presence of a dense anomaly extending at depth with density contrasts ranging from [-0.02, 0.05] g/cc. The negative density contrasts appear to be localized around the main positive anomaly, likely due to the smoothness constraint and the lack of `bound` constraints. Some smaller "fuzzy" anomalies are visible at the margins.  

As an alternative solution, the compact model (iteration 25) recovers a well-defined dense body within a mostly uniform background. Density contrasts have substantially increased in the range of [0.0, 0.56] g/cc.


## Summary

- We have inverted publicly available data over the Forrestania geological province.

- Our results suggest the presence of different geological units with distinct density and magnetic susceptibility signatures.
